# 0. Enable GPU in Kaggle

In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA Available: True
Device: Tesla T4


# 1. Environment & Determinism

In [2]:
import os, time, json, random
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda")
print("Using:", torch.cuda.get_device_name(0))

Using: Tesla T4


# 2. Load Clean Metadata

In [3]:
df = pd.read_csv("/kaggle/input/datasets/saifewu/py-crackdb-clean-dataset-metadata/clean_dataset_metadata.csv")
print(df['label'].value_counts())

label
With crack       369
Without crack    200
Name: count, dtype: int64


# 3. Optimized Transforms

In [4]:
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 4. High-Performance Dataset

In [5]:
from PIL import Image

class CrackDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.label_map = {"With crack": 1, "Without crack": 0}
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        label = self.label_map[row["label"]]
        return self.transform(img), torch.tensor(label, dtype=torch.float32)

# 5. Model Factory

In [6]:
def get_model(name):
    if name == "resnet50":
        model = models.resnet50(weights="IMAGENET1K_V1")
        model.fc = nn.Linear(model.fc.in_features, 1)

    elif name == "resnet101":
        model = models.resnet101(weights="IMAGENET1K_V1")
        model.fc = nn.Linear(model.fc.in_features, 1)

    elif name == "mobilenet_v2":
        model = models.mobilenet_v2(weights="IMAGENET1K_V1")
        model.classifier[1] = nn.Linear(model.last_channel, 1)

    elif name == "efficientnet_b0":
        model = models.efficientnet_b0(weights="IMAGENET1K_V1")
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)

    elif name == "vit_b_16":
        model = models.vit_b_16(weights="IMAGENET1K_V1")
        model.heads.head = nn.Linear(model.heads.head.in_features, 1)

    return model.to(device)

# 6. AMP Mixed Precision

In [7]:
scaler = torch.amp.GradScaler("cuda")

# 7. Training Loop

In [8]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            outputs = model(imgs).squeeze()
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
    
    return running_loss / len(loader)

# 8. Evaluation

In [9]:
def evaluate(model, loader):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)

            outputs = torch.sigmoid(model(imgs))
            outputs = outputs.view(-1)

            preds.extend(outputs.cpu().numpy().tolist())
            targets.extend(labels.numpy().tolist())

    preds_bin = (np.array(preds) > 0.5).astype(int)

    return {
        "accuracy": accuracy_score(targets, preds_bin),
        "precision": precision_score(targets, preds_bin, zero_division=0),
        "recall": recall_score(targets, preds_bin, zero_division=0),
        "f1": f1_score(targets, preds_bin, zero_division=0),
        "roc_auc": roc_auc_score(targets, preds)
    }

# 9. Multi-Ratio + Early Stopping + Scheduler

In [10]:
from sklearn.model_selection import train_test_split

RATIOS = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
results = []

for ratio in RATIOS:

    train_df, test_df = train_test_split(
        df, train_size=ratio,
        stratify=df["label"], random_state=SEED
    )

    train_df, val_df = train_test_split(
        train_df, test_size=0.1,
        stratify=train_df["label"], random_state=SEED
    )

    train_loader = DataLoader(
        CrackDataset(train_df, train_transform),
        batch_size=32, shuffle=True,
        num_workers=0, pin_memory=True
    )

    val_loader = DataLoader(
        CrackDataset(val_df, val_transform),
        batch_size=32, shuffle=False,
        num_workers=0, pin_memory=True
    )

    test_loader = DataLoader(
        CrackDataset(test_df, val_transform),
        batch_size=32, shuffle=False,
        num_workers=0, pin_memory=True
    )

    for model_name in ["resnet50","resnet101",
                       "mobilenet_v2",
                       "efficientnet_b0",
                       "vit_b_16"]:

        model = get_model(model_name)
        optimizer = optim.AdamW(model.parameters(), lr=3e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
        criterion = nn.BCEWithLogitsLoss()

        best_auc = 0
        patience = 5
        counter = 0

        start_time = time.time()

        for epoch in range(50):
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
            val_metrics = evaluate(model, val_loader)
            scheduler.step()

            if val_metrics["roc_auc"] > best_auc:
                best_auc = val_metrics["roc_auc"]
                counter = 0
                torch.save(model.state_dict(),
                           f"/kaggle/working/{model_name}_{ratio}_best.pth")
            else:
                counter += 1
                if counter >= patience:
                    break

        train_time = time.time() - start_time

        model.load_state_dict(torch.load(
            f"/kaggle/working/{model_name}_{ratio}_best.pth"))

        test_metrics = evaluate(model, test_loader)
        test_metrics["model"] = model_name
        test_metrics["ratio"] = ratio
        test_metrics["train_time"] = train_time

        results.append(test_metrics)

        torch.cuda.empty_cache()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s]


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:00<00:00, 199MB/s]


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 121MB/s]


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 134MB/s] 


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:01<00:00, 239MB/s]


# 10. Save Structured Results

In [11]:
results_df = pd.DataFrame(results)
results_df.to_csv("/kaggle/working/task2_gpu_results.csv", index=False)
results_df.sort_values("roc_auc", ascending=False).head()

,accuracy,precision,recall,f1,roc_auc,model,ratio,train_time
1,0.929825,0.902439,1.000000,0.948718,1.0,resnet101,0.9,33.893218
3,0.982456,1.000000,0.972973,0.986301,1.0,efficientnet_b0,0.9,33.701410
2,0.982456,1.000000,0.972973,0.986301,1.0,mobilenet_v2,0.9,36.661913
7,0.991228,1.000000,0.986486,0.993197,1.0,mobilenet_v2,0.8,25.872694
5,1.000000,1.000000,1.000000,1.000000,1.0,resnet50,0.8,27.626568
